In [1]:
import tkinter as tk
from tkinter import messagebox
import customtkinter as ctk
from tkinter import PhotoImage, ttk
from PIL import Image, ImageTk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
from scipy.stats import norm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from plotly.offline import plot
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from itertools import chain
from sklearn import preprocessing
import copy as copy
import os

In [2]:
Data_input = pd.read_excel('DataV1.xlsx')
Data_output = pd.read_excel('DataV1.xlsx',sheet_name='LungeTime')


y = Data_output['Time post'][Data_input['Reconstruction']=='Hamstring']
WorkingData = Data_input[Data_input['Reconstruction']=='Hamstring']

WorkingData = WorkingData.iloc[: , :-1]

WorkingData = pd.get_dummies(WorkingData, columns = ['Sex'])
WorkingData = WorkingData.drop(['ID','WeightPost'],axis=1)

CopyWorkingData = copy.deepcopy(WorkingData)


#WorkingData[['Age','Height','Weight']] = np.log(WorkingData[['Age','Height','Weight']])
WorkingData['Sex_F'] = WorkingData['Sex_F'].astype(int)
WorkingData['Sex_M'] = WorkingData['Sex_M'].astype(int)
WorkingData['Height'] =  preprocessing.normalize([WorkingData['Height']*100])[0]
WorkingData['Age'] = preprocessing.normalize([WorkingData['Age']])[0]
WorkingData['Weight'] = preprocessing.normalize([WorkingData['Weight']])[0]

X = WorkingData


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=2)


In [3]:
model = Sequential()
model.add(Dense(4, input_dim=5, activation='sigmoid'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='SGD', loss='mean_squared_error')

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

history = model.fit(X, y, epochs=100, batch_size=10, verbose=0)

print(model.summary())
predictions = model.predict(X_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 24        
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 137ms/step


In [64]:
canvasBoxplot = None
canvasLine = None
canvasHist = None
expand_button = None
bottom_frame_label = None
bottom_frameHist_label = None
Switch = None
counter = 0
def validate_numeric_input(input_text):
        return input_text.isdigit() or (input_text == "")

LeeftijdVerdeling = pd.read_csv('Age.csv')
LungeData =pd.read_excel('data lunge time.xlsx')
LungeJong = LungeData[LungeData['Age'] <= 25]['Lunge time']
LungeOud = LungeData[LungeData['Age'] > 25]['Lunge time']

def run_button_clicked():
    global canvasBoxplot, expand_button, canvasLine, canvasHist, bottom_frame_label, bottom_frameHist_label,Switch
    # als dit er is dan patient data toevoegen aan GUI
    Switch = tk.Button(root, text="<>", bg=button_color, fg="white", font=("Arial", 14),height= 1, width=6, command=Switch_button_click)
    Switch.place(relx=0.525, rely=0.875, anchor='center')
    def expand_button_clicked():
        global expand_button


        image_path = f"mri {pnumber}.jpg"

        # Create a new window
        expand_window = tk.Toplevel(root)

        # Load the image
        img = Image.open(image_path)
        img = img.resize((750, 750), Image.ANTIALIAS)  # Resize the image as needed

        # Convert the image to Tkinter PhotoImage
        img_tk = ImageTk.PhotoImage(img)

        # Create a label to display the image
        label = tk.Label(expand_window, image=img_tk)
        label.image = img_tk
        label.pack()
    
    # Variables from input
    gender = gender_var.get()
    weightstr = weight_var.get()
    lengthstr = length_var.get()
    agestr = age_var.get()
    pnumber = pnumber_var.get()

    # Check voor nummers
    if not (weightstr.replace('.', '', 1).isdigit() and lengthstr.replace('.', '', 1).isdigit() and agestr.isdigit()):
        messagebox.showerror("Error", "Please enter numeric values for Weight, Length, and Age.")
        return
    gender = gender_var.get()
    weight = float(weight_var.get()) / np.linalg.norm(CopyWorkingData['Weight'], keepdims=True)
    length = float(length_var.get()) / np.linalg.norm(CopyWorkingData['Height'], keepdims=True)
    age = float(age_var.get()) / np.linalg.norm(CopyWorkingData['Age'], keepdims=True)
    genderInput = [0,1]
    patient_name = name_var.get()
    pnumber = pnumber_var.get()
    
    # format de data voor de input van NN, waarschijnlijk dit aanpassen in juiste format
    if gender == "F":
        genderInput = [1,0]
    
    input_data = [float(age),float(weight), float(length)]
    input_data = input_data + genderInput
    Input = np.array([input_data],dtype='f')
        
    # prediction maken van de patient
    try:
        prediction = model.predict(Input)
        Output = np.round(prediction[0][0],2)
        #output label van NN maken
        if Output > 2.20:
            output_label.config(text=f"Lunge time (in s): \n {Output:.4f} \n \n Average lunge time: \n 1.11-2.20 s \n \n Our recomendation:\nDo not perate")
        elif Output <= 2.20:
            output_label.config(text=f"Lunge time (in s): \n {Output:.4f} \n \n Average lunge time: \n  1.11-2.20 s \n \n Our recomendation:\nOperate")
        
    except Exception as e:
        print(f"Error during prediction: {e}")
        messagebox.showerror("Error", "An error occurred during prediction")
        
    # check of er een bestand bekend is met het volgende patient number:


    def check_file_existence(directory_path, target_filename):
        global expand_button
        files = os.listdir(directory_path)


        if (f'Patient data {target_filename}.xlsx') in files:
            df_patient = pd.read_excel(f'Patient data {pnumber}.xlsx')
            behandeling = df_patient['Previous visit'].values
            behandelingen = '\n'.join(map(str, behandeling))   
            behandeling_date = df_patient['Visit date'].dt.strftime('%d-%m-%Y').values
            behandelingen_date = '\n'.join(map(str, behandeling_date)) 
            output_label_patient.config(text=f"Patient Name: \n{patient_name} \n\nPrevious Treatments: \n{behandelingen}")
            output_label_patient_data.config(text=f"\n\n\nDate: \n{behandelingen_date}")
            image_path = f"mri {pnumber}.jpg"
            img = Image.open(image_path)
            img = img.resize((252, 252), Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(img) 
            label = tk.Label(top_right3_frame, image=photo)
            label.image = photo
            label.pack()

            # Expand image button
            expand_button = tk.Button(root, text="+", bg=button_color, fg="white", font=("Arial", 14), height=1, width=2, command=expand_button_clicked)
            expand_button.place(relx=0.94, rely=0.075, anchor="center")

        else:
            output_label_patient.config(text=f"Patient Name: \n{patient_name} \n\nPatient does not\nappear in\ndatabase")
    
    target_filename = pnumber
    directory_path = r"C:\Users\joris\Downloads\Dashboard Final\Dashboard Final"  # Vervang met het daadwerkelijke pad naar je map

    check_file_existence(directory_path, target_filename)
    
    if canvasLine:
        canvasLine.get_tk_widget().destroy()
        canvasLine = None
    figure = Figure(figsize=(5, 1), dpi=100)
    LineGraph = figure.add_subplot(1, 1, 1)

    HeightAverage = 184
    HeightStd = 7
    
    x_axis = np.arange(HeightAverage-3*HeightStd, HeightAverage+3*HeightStd, 0.01) 
    x_list = np.arange(HeightAverage-3*HeightStd, HeightAverage+3*HeightStd,HeightStd ) 
    # Plot the new data
    LineGraph.plot(x_axis, norm.pdf(x_axis, HeightAverage, HeightStd),color='blue')
    
    for i in x_list:
        LineGraph.vlines(x=i,ymax = norm.pdf(i,HeightAverage,HeightStd),ymin = 0,colors = 'blue')
        
    LineGraph.vlines(x=float(lengthstr),ymax = norm.pdf(float(lengthstr),HeightAverage,HeightStd),ymin = 0,colors = 'red')
    LineGraph.set_title('Distrubition of Height in The Netherlands')
    LineGraph.set_xlabel('Height (cm)')
    LineGraph.set_ylabel('Probability')
    canvasLine = FigureCanvasTkAgg(figure, master=bottom_frame)
    canvasLine.draw()
    canvasLine.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
    canvasLine._tkcanvas.pack(side = tk.TOP, fill = tk.BOTH, expand = True)
    root.canvas = canvasLine
                        
    FigureV2 = Figure(figsize=(5, 4), dpi=100)
    Boxplot = FigureV2.add_subplot(1, 1, 1)

    if float(agestr) > 25:
        Boxplot.boxplot(LungeOud, medianprops = dict(color = "blue", linewidth = 1.5))
        Boxplot.set_title(f'Standard lunge time for people aged 25-34')
    else:
        Boxplot.boxplot(LungeJong, medianprops = dict(color = "blue", linewidth = 1.5))
        Boxplot.set_title(f'Standard lunge time for people aged 15-24')
    
    Boxplot.plot(1,Output,'r.', alpha=0.8,marker='*',linewidth=2)
    colors = {'Height':'red', 'Mean':'blue'}
    labels = list(colors.keys())
    handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
    Boxplot.legend(handles, labels)
    Boxplot.set_xticks([])
    
    Boxplot.set_ylabel('Predicted Lunge Time (s)')

    
    # Clear the existing boxplot canvas
    if canvasBoxplot:
        canvasBoxplot.get_tk_widget().destroy()

    # Create a new canvas for boxplot and pack it
    canvasBoxplot = FigureCanvasTkAgg(FigureV2, master=left_frame)
    canvasBoxplot.draw()
    canvasBoxplot.get_tk_widget().grid(row=0, column=0, sticky="we")
    root.canvas = canvasBoxplot

def Switch_button_click():
    global canvasHist, canvasLine, counter
    if canvasHist:
        canvasHist.get_tk_widget().destroy()
        canvasHist = None  
    if canvasLine:
        canvasLine.get_tk_widget().destroy()
        canvasLine = None 
    gender = gender_var.get()
    weightstr = weight_var.get()
    lengthstr = length_var.get()
    agestr = age_var.get()
    pnumber = pnumber_var.get()
    if (counter%2)==0:
        figureV3 = Figure(figsize=(8, 4), dpi=100)
        Histogram = figureV3.add_subplot(1, 1, 1)
        # Plot the new data
        LeeftijdGroepering = []
        LeeftijdGroeperingStr = []
        Colors = []
        S = 0
        for i in range(len(LeeftijdVerdeling['Men'])):
            S += float(LeeftijdVerdeling['Men'][i].replace(" ",""))
            if (i % 5) == 0 and i > 0:
                S = S / 1000
                LeeftijdGroepering.append(S)
                S = 0
                Str = f'{i-5}-{i}'
                LeeftijdGroeperingStr.append(Str)
                if float(agestr) in np.arange(i-5,i,1):
                    Colors.append('red')
                else:        
                    Colors.append('blue')

        xaxis = np.arange(0, len(LeeftijdGroepering))  

        Histogram.bar(xaxis, LeeftijdGroepering, color=Colors)
        Histogram.set_xticks(xaxis)
        Histogram.set_xticklabels(LeeftijdGroeperingStr, rotation=45, fontdict={'size': 7})
        Histogram.set_yticks(np.arange(0, max(LeeftijdGroepering), 200))

        Histogram.set_ylabel('Amount of People in 1000')
        Histogram.set_xlabel('Age Groups')
        Histogram.set_title('Distribution of Age Groups')



        canvasHist = FigureCanvasTkAgg(figureV3, master=bottom_frame)
        canvasHist.draw()
        canvasHist.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
    else:
        figure = Figure(figsize=(5, 1), dpi=100)
        LineGraph = figure.add_subplot(1, 1, 1)

        HeightAverage = 184
        HeightStd = 7

        x_axis = np.arange(HeightAverage-3*HeightStd, HeightAverage+3*HeightStd, 0.01) 
        x_list = np.arange(HeightAverage-3*HeightStd, HeightAverage+3*HeightStd,HeightStd ) 
        # Plot the new data
        LineGraph.plot(x_axis, norm.pdf(x_axis, HeightAverage, HeightStd),color='blue')

        for i in x_list:
            LineGraph.vlines(x=i,ymax = norm.pdf(i,HeightAverage,HeightStd),ymin = 0,colors = 'blue')

        LineGraph.vlines(x=float(lengthstr),ymax = norm.pdf(float(lengthstr),HeightAverage,HeightStd),ymin = 0,colors = 'red')
        LineGraph.set_title('Distrubition of Height in The Netherlands')
        LineGraph.set_xlabel('Height (cm)')
        LineGraph.set_ylabel('Probability')
        canvasLine = FigureCanvasTkAgg(figure, master=bottom_frame)
        canvasLine.draw()
        canvasLine.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        canvasLine._tkcanvas.pack(side = tk.TOP, fill = tk.BOTH, expand = True)
        root.canvas = canvasLine
    counter += 1
    
    
    
def reset_button_clicked():
    global canvasBoxplot, expand_button, canvasLine, canvasHist, bottom_frame_label, bottom_frameHist_label, Switch
    # Clear input fields
    gender_var.set(" ")
    name_var.set("")
    weight_var.set("")
    length_var.set("")
    age_var.set("")
    pnumber_var.set("")

    # Clear output labels
    output_label.config(text="")
    output_label_patient.config(text="")
    output_label_patient_data.config(text="")

    # Clear existing boxplot
    if canvasBoxplot:
        canvasBoxplot.get_tk_widget().destroy()
        canvasBoxplot = None
    
        
    if canvasLine:
        canvasLine.get_tk_widget().destroy()
        canvasLine = None
        
    if canvasHist:
        canvasHist.get_tk_widget().destroy()
        canvasHist = None
        
    for widget in top_right3_frame.winfo_children():
        if isinstance(widget, tk.Label):
            widget.destroy()
            
    if expand_button:
        expand_button.destroy()
        expand_button = None
    
    if Switch:
        Switch.destroy()
        Switch = None
        
    if bottom_frame_label:
        bottom_frame_label.destroy()
        bottom_frame_label = None
        
    if bottom_frameHist_label:
        bottom_frameHist_label.destroy()
        bottom_frameHist_label = None

        
# main window
root = tk.Tk()
root.title("Python Dashboard")

# size
root.geometry("1500x900")
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)
# kleuren, hex en namen
background_color = "#386bd1" 
frame_color = "#b6c5e3"  
button_color = "grey"  
label_color = "black"
label_text_color = "black"

# frames voor de gebieden in de GUI
root.configure(bg=background_color)

left_frame = tk.Frame(root, bg=background_color)
left_frame.place(relx=0.59, rely=0.4, relwidth=0.4, relheight=0.5)

top_left_frame = tk.Frame(root, bg=frame_color)
top_left_frame.place(relx=0.05, rely=0.05, relwidth=0.25, relheight=0.3)

top_right_frame = tk.Frame(root, bg=frame_color)
top_right_frame.place(relx=0.412, rely=0.05, relwidth=0.14, relheight=0.3)

top_right2_frame = tk.Frame(root, bg=frame_color)
top_right2_frame.place(relx=0.565, rely=0.05, relwidth=0.13, relheight=0.3)

top_right2b_frame = tk.Frame(root, bg=frame_color)
top_right2b_frame.place(relx=0.69, rely=0.05, relwidth=0.1, relheight=0.3)

top_right3_frame = tk.Frame(root, bg=background_color)
top_right3_frame.place(relx=0.765, rely=0.05, relwidth=0.2, relheight=0.3)

bottom_frame = tk.Frame(root, bg=background_color)
bottom_frame.place(relx=0.05, rely=0.375, relwidth=0.5, relheight=0.5)


# gender binary variable
gender_var = tk.StringVar()
gender_label = tk.Label(top_left_frame, text="Sex:", bg=frame_color, fg=label_color, font=("Arial", 14))
gender_label.grid(row=0, column=0, padx=10, pady=10)

male_button = tk.Radiobutton(top_left_frame, text="M", variable=gender_var, value="M", bg=frame_color, fg=label_color, font=("Arial", 12))
male_button.grid(row=0, column=1, padx=(10, 2), pady=10, columnspan=1)

female_button = tk.Radiobutton(top_left_frame, text="F", variable=gender_var, value="F", bg=frame_color, fg=label_color, font=("Arial", 12))
female_button.grid(row=0, column=2, padx=(2, 10), pady=10, columnspan=1)

gender_var.set(" ")


# Entry boxes voor variable: weight, name, length, and age
name_var = tk.StringVar()
name_label = tk.Label(top_left_frame, text="Name:", bg=frame_color, fg=label_color, font=("Arial", 12))
name_label.grid(row=1, column=0, padx=10, pady=6)
name_entry = tk.Entry(top_left_frame, textvariable=name_var, font=("Arial", 11))
name_entry.grid(row=1, column=1, padx=10, pady=6)

validate_cmd = (root.register(validate_numeric_input), '%P')

#deze toegevoegd, patient numer, aangepast
pnumber_var = tk.StringVar()
pnumber_label = tk.Label(top_left_frame, text="Patient Number:", bg=frame_color, fg=label_color, font=("Arial", 12))
pnumber_label.grid(row=2, column=0, padx=10, pady=6)
pnumber_entry = tk.Entry(top_left_frame, textvariable=pnumber_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
pnumber_entry.grid(row=2, column=1, padx=10, pady=6)

weight_var = tk.StringVar()
weight_label = tk.Label(top_left_frame, text="Weight (kg):", bg=frame_color, fg=label_color, font=("Arial", 12))
weight_label.grid(row=3, column=0, padx=10, pady=6)
weight_entry = tk.Entry(top_left_frame, textvariable=weight_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
weight_entry.grid(row=3, column=1, padx=10, pady=6)

length_var = tk.StringVar()
length_label = tk.Label(top_left_frame, text="Height (cm):", bg=frame_color, fg=label_color, font=("Arial", 12))
length_label.grid(row=4, column=0, padx=10, pady=6)
length_entry = tk.Entry(top_left_frame, textvariable=length_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
length_entry.grid(row=4, column=1, padx=10, pady=6)

age_var = tk.StringVar()
age_label = tk.Label(top_left_frame, text="Age (years):", bg=frame_color, fg=label_color, font=("Arial", 12))
age_label.grid(row=5, column=0, padx=10, pady=6)
age_entry = tk.Entry(top_left_frame, textvariable=age_var, font=("Arial", 11), validate="key", validatecommand=validate_cmd)
age_entry.grid(row=5, column=1, padx=10, pady=6)

# Display for lunge time
output_label = tk.Label(top_right_frame, text="", bg=frame_color, fg=label_text_color, font=("Arial", 14))
output_label.pack(anchor='n', padx=10, pady=10)

# patient data
output_label_patient = tk.Label(top_right2_frame, text="", bg=frame_color, fg=label_text_color, font=("Arial", 13), justify="left")
output_label_patient.pack(anchor='w', padx=10, pady=10)

output_label_patient_data = tk.Label(top_right2b_frame, text="", bg=frame_color, fg=label_text_color, font=("Arial", 13), justify="left")
output_label_patient_data.pack(anchor='w', padx=10, pady=10)

# Run button
run_button = tk.Button(root, text="Analyze", bg=button_color, fg="white", font=("Arial", 25),height= 1, width=6, command=run_button_clicked)
run_button.place(relx=0.355, rely=0.1, anchor='center')

# Reset button
Reset_button = tk.Button(root, text="Reset", bg=button_color, fg="white", font=("Arial", 25),height= 1, width=6, command=reset_button_clicked)
Reset_button.place(relx=0.355, rely=0.25, anchor='center')


root.mainloop()

1/1 [==============================] - 0s 19ms/step


C:\Users\joris\AppData\Local\Temp\ipykernel_14164\3106702556.py:157: UserWarning:

marker is redundantly defined by the 'marker' keyword argument and the fmt string "r." (-> marker='.'). The keyword argument will take precedence.



1/1 [==============================] - 0s 26ms/step


C:\Users\joris\AppData\Local\Temp\ipykernel_14164\3106702556.py:157: UserWarning:

marker is redundantly defined by the 'marker' keyword argument and the fmt string "r." (-> marker='.'). The keyword argument will take precedence.

